# 생성적 적대 신경망(Generative Adversarial Networks)
- 가상의 이미지를 생성하는 알고리즘
- 진짜 같은 가짜를 만들기 위해 GAN 알고리즘 내부에서는 ‘적대적’인 경합을 진행
- 생성자(Generator) : 가상의 이미지를 만들어 내는 공장
- 처음엔 랜덤한 픽셀 값으로 채워진 가짜 이미지로 시작해서 판별자의 판별 결과에 따라 지속적으로 업데이트
- = 판별자(Discriminator)


# 페이스북의 AI 연구팀- DCGAN(Deep Convolutional GAN)
- 1) DCGAN은 생성자가 가짜 이미지를 만들 때, 컨볼루션 신경망(CNN)을 이용
- 컨볼루션 신경망은 앞서 나온 것과 조금 차이
- 옵티마이저(optimizer)를 사용하는 최적화 과정이나 컴파일하는 과정이 없다 => 판별과 학습이 이곳 생성자에서 일어나는 게 아니기 때문
- 2) 일부 매개 변수를 삭제하는 풀링(pooling) 과정이 없고, 대신 앞 장에서 배운 패딩(padding) 과정이 포함 (빈곳을 채워서 같은 크기로 맞추)
- 3) 배치 정규화(Batch Normalization) : 입력 데이터의 평균이 0, 분산이 1이 되도록 재배치, 다음 층으로 입력될 값을 일정하게 재배치하는 역할=> 층의 개수가 늘어나도 안정적인 학습을 진행. => keras BatchNormalization() 
- 4) 생성자의 활성화 함수로는 ReLU() 함수를 쓰고 판별자로 넘겨 주기 직전에는 tanh() 
- tanh() 함수를 쓰면 출력되는 값을 -1.0~+1.0 사이로
- 판별자에 입력될 MNIST 손글씨의 픽셀 범위도 -1~1로 맞추면 판별 조건이 모두 갖춰집

In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import os 
if not os.path.exists("./gan_images"):
    os.makedirs("./gan_images")

In [4]:
# 생성자 모델
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))
# 128은 임의로 정한 노드의 수
# Input_dim=100은 100차원 크기의 랜덤 벡터를 준비해 집어넣
# 7*7 이미지의 최초 크기 (UpSampling2D() =>이미지의 가로, 세로 크기를 2배씩 늘려 => 14x14 => 28x28)
generator.add(BatchNormalization()) 
# 데이터의 배치를 정규 분포로 만드는 배치 정규화
generator.add(Reshape((7, 7, 128)))
# 컨볼루션 레이어가 받아들일 수 있는 형태로 바꾸 = Conv2D() 함수의 input_shape 부분에 들어갈 형태로 정해
generator.add(UpSampling2D()) # 14x14 
generator.add(Conv2D(64, kernel_size=5, padding='same'))
# 커널 크기를 3 => 3x3 크기의 마스크, 
# padding = ‘same’ => 모자라는 부분은 자동으로 0이 채워
generator.add(BatchNormalization())
# 데이터의 배치를 정규 분포로 만드는 배치 정규화
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D()) # 28x28
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh'))
# 한 번 더 컨볼루션 과정을 거친 후 판별자로 값을 넘길 준비

In [5]:
generator.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
batch_normalization_2 (Batch (None, 6272)              25088     
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        204864    
_________________________________________________________________
batch_normalization_3 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 64)       

In [6]:
# 판별자(생성자에서 넘어온 이미지가 가짜인지 진짜인지를 판별(discriminator)) 모델
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=5, strides=2, input_shape=(28,28,1), padding="same"))
# 노드의 수는 각각 64, 128개, 커널 크기는 5, 5x5 마스크
# stride는 마스크를 몇 칸씩 이동시킬지 => 가로, 세로 크기가 더 줄어 들어 새로운 특징을 뽑아주는 효과
# (드롭아웃이나 풀링처럼 새로운 필터를 적용한 효과)
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer='adam')
# 로스 함수(binary_crossentropy)와 최적화 함수(adam)를 써서 판별자에 필요한 준비를 마무리
discriminator.trainable = False

# 주의할 점은 이 판별자는 진짜인지 가짜인지 판별만 해 줄 뿐, 자기 자신이 학습을 해서는 안 된다 
# 판별자가 얻은 가중치는 판별자 자신이 학습하는 데 쓰이는 게 아니라 => 생성자로 넘겨 주어 생성자가 업데이트된 이미지를 만들도록 해야
# 따라서 판별자를 만들 때는 가중치를 저장하는 학습 기능을 꺼주어야

In [7]:
discriminator.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
activation_3 (Activation)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)             

In [8]:
# 생성자와 판별자 모델을 연결시키는 gan 모델 만들기 
ginput = Input(shape=(100,)) # 100 noise 
dis_output = discriminator(generator(ginput))
gan = Model(ginput, dis_output)

In [9]:
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [10]:
gan.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 28, 28, 1)         865281    
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 212865    
Total params: 1,078,146
Trainable params: 852,609
Non-trainable params: 225,537
_________________________________________________________________


In [15]:
# 신경망을 실행시키는 함수 만들기
def gan_train(epoch, batch_size, saving_interval):
    (X_train, _), (_, _) = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
    # MNIST 데이터 불러오기
    # 앞서 불러온 MNIST를 다시 이용, 테스트 과정은 필요없고 이미지만 사용할 것이기 때문에 X_train만 호출
    X_train = (X_train - 127.5) / 127.5 
    # 127.5를 빼준 뒤 127.5로 나눠서 -1~1사이의 값으로 바꿈
    true = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for i in range(epoch):
        # 실제 데이터를 판별자에 입력
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]  
        d_loss_real = discriminator.train_on_batch(imgs, true)
     
        # 가상 이미지를 판별자에 입력
        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
 
        # 판별자와 생성자의 오차 계산
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        g_loss = gan.train_on_batch(noise, true)
 
        print('epoch:%d' % i, ' d_loss:%.4f' % d_loss, ' g_loss:%.4f' % g_loss)
        
        if i % saving_interval == 0:
              #r, c = 5, 5
              noise = np.random.normal(0, 1, (25, 100))
              gen_imgs = generator.predict(noise)

              # Rescale images 0 - 1
              gen_imgs = 0.5 * gen_imgs + 0.5

              fig, axs = plt.subplots(5, 5)
              count = 0
              for j in range(5):
                  for k in range(5):
                      axs[j, k].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                      axs[j, k].axis('off')
                      count += 1
              fig.savefig("gan_images/gan_mnist_%d.png" % i)
    

In [ ]:
# 중간 과정을 이미지로 저장하는 부분. 정해진 인터벌만큼 학습되면 그때 만든 이미지를 gan_images 폴더에 저장하라는 뜻. 이 코드는 본 장의 주된 목표와는 관계가 없어서 소스 코드만 소개한다

In [16]:
# 4,000번 반복되고(+1을 하는 것에 주의), 배치 크기는 32, 200번마다 결과가 저장됨
gan_train(4001, 32, 200)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: No module named 'tensorflow_core.estimator'
epoch:0  d_loss:0.6940  g_loss:0.6164
epoch:1  d_loss:0.4393  g_loss:0.2783
epoch:2  d_loss:0.4497  g_loss:0.0702
epoch:3  d_loss:0.4808  g_loss:0.0294
epoch:4  d_loss:0.5335  g_loss:0.0380
epoch:5  d_loss:0.4841  g_loss:0.1252
epoch:6  d_loss:0.4415  g_loss:0.3579
epoch:7  d_loss:0.4929  g_loss:0.4646
epoch:8  d_loss:0.5242  g_loss:0.4119
epoch:9  d_loss:0.4955  g_loss:0.4283
epoch:10  d_loss:0.4532  g_loss:0.4794
epoch:11  d_loss:0.3861  g_loss:0.7155
epoch:12  d_loss:0.3459  g_loss:1.1948
epoch:13  d_loss:0.2968  g_loss:1.8312
epoch:14  d_loss:0.2592  g_loss:2.5049
epo

C:\Users\asus\anaconda3\envs\django3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
